<a href="https://colab.research.google.com/github/SohamSen21/BCS_task/blob/main/Sherlock_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import tensorflow and other necessary libraries
import tensorflow as tf
import numpy as np
import os
import time
import requests

In [ ]:
path_to_file = tf.keras.utils.get_file('sherlock.txt', 'https://assets.datacamp.com/production/repositories/3937/datasets/213ca262bf6af12428d42842848464565f3d5504/sherlock.txt')

6488666/6488666 [==============================] - 1s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 6488666 characters


In [ ]:
# Take a look at the first 10000 characters in text
print(text[:10000])

The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan Doyle)

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**eBooks Readable By Both Humans and By Computers, Since 1971**

*****These eBooks Were Prepared By Thousan

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

93 unique characters


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[66, 67, 68, 69, 70, 71, 72], [89, 90, 91]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(6488666,), dtype=int64, numpy=array([55, 73, 70, ..., 84, 63,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
e
 
P
r
o
j
e
c


In [ ]:
seq_length = 100


In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'h' b'e' b' ' b'P' b'r' b'o' b'j' b'e' b'c' b't' b' ' b'G' b'u'
 b't' b'e' b'n' b'b' b'e' b'r' b'g' b' ' b'E' b'B' b'o' b'o' b'k' b' '
 b'o' b'f' b' ' b'T' b'h' b'e' b' ' b'A' b'd' b'v' b'e' b'n' b't' b'u'
 b'r' b'e' b's' b' ' b'o' b'f' b' ' b'S' b'h' b'e' b'r' b'l' b'o' b'c'
 b'k' b' ' b'H' b'o' b'l' b'm' b'e' b's' b'\n' b'b' b'y' b' ' b'S' b'i'
 b'r' b' ' b'A' b'r' b't' b'h' b'u' b'r' b' ' b'C' b'o' b'n' b'a' b'n'
 b' ' b'D' b'o' b'y' b'l' b'e' b'\n' b'(' b'#' b'1' b'5' b' ' b'i' b'n'
 b' ' b'o' b'u'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in ou'
b'r series by Sir Arthur Conan Doyle)\n\nCopyright laws are changing all over the world. Be sure to check'
b' the\ncopyright laws for your country before downloading or redistributing\nthis or any other Project G'
b'utenberg eBook.\n\nThis header should be the first thing seen when viewing this Project\nGutenberg file.'
b'  Please do not remove it.  Do not change or edit the\nheader without written permission.\n\nPlease read'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in o'
Target: b'he Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in ou'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24064     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  96350     
                                                                 
Total params: 4,058,718
Trainable params: 4,058,718
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([28, 30, 51, 59, 64, 56, 72,  2, 12, 19, 72, 49, 93, 64, 47, 68, 27,
       71, 40, 28, 13, 49, 38, 82,  2, 51, 92, 83, 70, 26, 64, 76, 45, 64,
       31,  0, 21, 74,  5, 35, 41, 35, 60, 18, 81,  4, 20, 44, 65, 10, 69,
       38, 63, 92, 38, 28, 75, 73, 31, 17,  1, 58, 70, 43, 72, 43, 59, 80,
       60, 31, 41, 18,  6, 89, 51, 34, 72, 20, 80, 73,  3, 52, 32, 31, 43,
       72, 42, 25,  8, 89,  6, 43, 72, 12, 17,  4, 72, 12, 58, 47])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'd London street. Looking over his shoulder, I saw that on the pavement opposite there stood a large '

Next Char Predictions:
 b'9;PX^Ug\n)0gN~^Lc8fE9*NCq\nP|re7^kJ^<[UNK]2i"@F@Y/p!1I_\'dC]|C9jh<.\tWeHgHXoY<F/#xP?g1oh Q=<HgG6%x#Hg).!g)WL'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 94)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.543341, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

94.00436

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
1003/1003 [==============================] - 64s 58ms/step - loss: 1.8468
Epoch 2/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.3185
Epoch 3/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.2218
Epoch 4/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.1716
Epoch 5/20
1003/1003 [==============================] - 56s 54ms/step - loss: 1.1367
Epoch 6/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.1099
Epoch 7/20
1003/1003 [==============================] - 57s 54ms/step - loss: 1.0884
Epoch 8/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.0705
Epoch 9/20
1003/1003 [==============================] - 56s 54ms/step - loss: 1.0563
Epoch 10/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.0447
Epoch 11/20
1003/1003 [==============================] - 57s 55ms/step - loss: 1.0361
Epoch 12/20
1003/1003 [==============================] - 56s 54

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Watson'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Watson's attempt to waist and the rest.

After the result of a mucous swallowy contraction of the first man
the entrance to questions and peasants and lifes of the band, the
so-called up the mouth, they were told as her days. But he heard the
officer into the road on his head.

"And here has no conception on their will. Yesterday Can
nerve me at the end of us "goodnesses "for me to see him."

"Call Him!" Napoleon replied to the pale eyebrows girly
hungry to the cart was of this to the people again appointed.





CHAPTER XV


Wither the presents chyless himself against thom were all muttered
just as if possible, indorsed some of good service and lack on and love him back
in which he had noticed that was had ridden before the ruge till the
present streets about in Mass.

On his cagn, following him for supposed to feed, the
evening. So now this is what the clouds even independent of it
askanal, a bettal light between the difficulties of
disgusting at Kalukhovo in a sense of knit, present